## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `<mongoimport -d uk_food -c establishments --jsonArray establishments.json>`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
pprint(mongo.list_database_names())

# 'ugh' ~ my poor computer

['admin', 'config', 'local']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
pprint(db.list_collection_names())

[]


In [6]:
# review the collections in our new database
# ^
# pretty sure I just did that

In [7]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

None


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
input = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [10]:
# Insert the new restaurant into the collection
establishments.insert_one(input)

InsertOneResult(ObjectId('66b1730b311d8893da0b86d8'), acknowledged=True)

In [11]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66b1730b311d8893da0b86d8'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# heh, lucked out with my first find_one exploration:
# 'BusinessType': 'Restaurant/Cafe/Canteen',
# 'BusinessTypeID': 1,
eg = {'BusinessName': 'Mrs Knotts Tea Room'}
id = {'BusinessTypeID', 'BusinessType'}

pprint(establishments.find_one(eg, id))

None


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
pf_name = {'BusinessName':'Penang Flavours'}
establishments.update_one(pf_name, {'$set':{ 'BusinessTypeID': 1}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one(pf_name, id))

# if you're particular, run this for all the deets:
# pprint(establishments.find_one(pf_name))

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66b1730b311d8893da0b86d8')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover = {'LocalAuthorityName': 'Dover'}
dovers = establishments.find(dover)

establishments.count_documents(dover)

# saving this for later...
# for whatever in dovers:
#    pprint(whatever)

0

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(dover)

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [17]:
# Check if any remaining documents include Dover
for whatever in dovers:
    pprint(whatever)

In [18]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('66b1730b311d8893da0b86d8'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
long = 'geocode.longitude'
slong = '$geocode.longitude'
lat = 'geocode.latitude'
slat = '$geocode.latitude'
establishments.update_many({}, [ {'$set':{ long : {'$toDouble': slong},  lat : {'$toDouble': slat} }   } ])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [ {'$set':{ 'RatingValue' : {'$toInt':'$RatingValue'} }   } ])

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Check that the coordinates and rating value are now numbers
results = {long, lat, 'RatingValue'}
tricky = establishments.find_one({}, results)
for dtype in tricky:
    pprint(dtype)
    pprint(type(tricky[dtype]))

# okay... halfway there

'_id'
<class 'bson.objectid.ObjectId'>
'geocode'
<class 'dict'>
'RatingValue'
<class 'NoneType'>


In [23]:
coordinates_only = {long, lat}
holy_cow = establishments.find_one({}, coordinates_only)
pprint(holy_cow)
    

{'_id': ObjectId('66b1730b311d8893da0b86d8'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}


In [24]:
for milk in holy_cow.keys():
    print(milk, type(holy_cow[milk]))
#    for whey in milk.keys():
#        pprint(whey, type(milk[whey]))

_id <class 'bson.objectid.ObjectId'>
geocode <class 'dict'>


In [25]:
# well, I'm giving up on the gocode paramater datatypes, here's a json with numbers

pprint(tricky)

{'RatingValue': None,
 '_id': ObjectId('66b1730b311d8893da0b86d8'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}
